In [1]:
import os
import sys

import json

import numpy as np
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report

import scarches as sca
from scarches.dataset.trvae.data_handling import remove_sparsity
import matplotlib.pyplot as plt

from lightning.pytorch.callbacks import ModelCheckpoint

import torch

from dotenv import load_dotenv

from lightning.pytorch.loggers import WandbLogger
import wandb

import session_info
import warnings
from pyprojroot.here import here

#plt.style.use(['science','nature','no-latex'])
dpi_fig_save = 300
sc.set_figure_params(dpi=100, dpi_save=dpi_fig_save, vector_friendly=True)

# Setting some parameters
warnings.filterwarnings("ignore")

from sklearn.model_selection import StratifiedKFold

overwriteData = True
overwriteFigures = True

# Set random seed
random_seed = 42

import warnings
warnings.filterwarnings('ignore')

import scvi
scvi.settings.dl_num_workers = 0
scvi.settings.seed = random_seed

#torch.set_float32_matmul_precision('high')
#torch.set_printoptions(precision=3, sci_mode=False, edgeitems=7)
# torch.multiprocessing.set_sharing_strategy('file_system')

%load_ext autoreload
%autoreload 2

/home/jupyter/conda_envs/scarches/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 captum (see https://github.com/pytorch/captum).


INFO: Seed set to 42


INFO:lightning.fabric.utilities.seed:Seed set to 42


In [2]:
assert load_dotenv()

In [3]:
workingDir = here('03_downstream_analysis/04_integration_with_annotation/')
workingDir

PosixPath('/home/jupyter/Inflammation-PBMCs-Atlas/03_downstream_analysis/03_scANVI_integration_with_annotation')

In [4]:
class CustomWandbLogger(WandbLogger):
    @property
    def save_dir(self):
        """Gets the save directory.

        Returns:
            The path to the save directory.

        """
        return self.experiment.dir

In [5]:
scvi.__version__

'1.1.2'

# Loading data


In [6]:
# Load the h5ad file
adata = sc.read_h5ad(here("03_downstream_analysis/04_integration_with_annotation/04_MAIN_geneUniverse_noRBCnPlatelets.h5ad"))#, 
                     #backed='r+', chunk_size=50000)


In [7]:
adata.obs['binned_age'] = adata.obs['binned_age'].astype(str)

#### Preparing scANVI training

In [8]:
scvi_model = sca.models.SCVI.load(here(f"{workingDir}/results/scVI_model_pretreined_noRBCnPlat/"), adata=adata) 

INFO     File                                                                                                      
         /home/jupyter/Inflammation-PBMCs-Atlas/03_downstream_analysis/04_integration_with_annotation/resu
         lts/scVI_model_pretreined_noRBCnPlat/model.pt already downloaded                                          


### Fine tuning with scANVI

**Parameters**

In [9]:
scANVI_trainer_kwargs = dict(
    n_samples_per_label = None,
    check_val_every_n_epoch = None,
    train_size = 0.8,
    validation_size = 0.2,
    shuffle_set_split = True,
    checkpointing_monitor = 'elbo_validation',
    early_stopping_monitor = 'reconstruction_loss_validation',
    early_stopping_patience = 2,
    early_stopping_min_delta=0.1,
    early_stopping = True,
    max_epochs = 1000,
)
# https://docs.scvi-tools.org/en/stable/api/reference/scvi.train.TrainingPlan.html#scvi.train.TrainingPlan
plan_kwargs = dict(
    lr = 5e-5,
    #reduce_lr_on_plateau = True
)
datasplitter_kwargs = dict(pin_memory=False)
scanvi_parameter_dict = scANVI_trainer_kwargs | plan_kwargs | datasplitter_kwargs

In [10]:
run_name = f"MAINobj_scANVI_fineTuning_lowLR_noRBCnPlat"
run_name

'MAINobj_scANVI_fineTuning_lowLR_noRBCnPlat'

In [11]:
scanvi_model = sca.models.SCANVI.from_scvi_model(scvi_model, unlabeled_category = "unknown")
scanvi_model

ScanVI Model with the following params: 
unlabeled_category: unknown, n_hidden: 256, n_latent: 30, n_layers: 4, dropout_rate: 0.1, dispersion: gene-batch, 
gene_likelihood: nb
Training status: Not Trained
Model's adata is minified?: False

In [12]:
logger = CustomWandbLogger(name = run_name, project='inflammation_atlas_R1_scANVI', config = scanvi_parameter_dict)

In [13]:
model_checkpoint = ModelCheckpoint(monitor='elbo_validation', every_n_epochs=1, save_last=True, save_top_k = -1, 
                                   dirpath=f"{workingDir}/results/scANVI_model_fineTuned_lowLR_noRBCnPlat_checkpoints/")

In [14]:
try:
    scanvi_model.train(logger=logger, plan_kwargs = plan_kwargs, datasplitter_kwargs=datasplitter_kwargs, 
                       enable_checkpointing=True, callbacks=[model_checkpoint], 
                       **scANVI_trainer_kwargs) #SAME parameter as scVI
    scanvi_model.save(here(f"{workingDir}/results/scANVI_model_fineTuned_lowLR_noRBCnPlat/"),
               overwrite = True, 
               save_anndata = False)
    scanvi_emb = scanvi_model.get_latent_representation(adata=adata)
    np.savez_compressed(file = str(here(f"{workingDir}/results/scANVI_model_fineTuned_lowLR_noRBCnPlat/scANVI_embedding.npz")), arr=scanvi_emb)
    
except Exception as e:
    print(f"An error occurred: {e}")
    scanvi_model.save(here(f"{workingDir}/results/scANVI_model_fineTuned_lowLR_noRBCnPlat_WITHERRORS/"), 
                      overwrite=True, 
                      save_anndata=False)

INFO     Training for 1000 epochs.                                                                                 


INFO: GPU available: True (cuda), used: True


INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True


INFO: TPU available: False, using: 0 TPU cores


INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores


...

Epoch 16/1000:   2%|▋                                            | 15/1000 [6:01:17<394:23:52, 1441.45s/it, v_num=jj32, train_loss_step=1.78e+3, train_loss_epoch=1.88e+3]

Epoch 16/1000:   2%|▋                                            | 16/1000 [6:25:29<394:56:18, 1444.90s/it, v_num=jj32, train_loss_step=1.78e+3, train_loss_epoch=1.88e+3]

Epoch 16/1000:   2%|▋                                            | 16/1000 [6:25:29<394:56:18, 1444.90s/it, v_num=jj32, train_loss_step=1.94e+3, train_loss_epoch=1.88e+3]

Epoch 16/1000:   2%|▋                                            | 16/1000 [6:25:29<395:08:10, 1445.62s/it, v_num=jj32, train_loss_step=1.94e+3, train_loss_epoch=1.88e+3]

Monitored metric reconstruction_loss_validation did not improve in the last 2 records. Best score: 1855.179. Signaling Trainer to stop.


In [15]:
wandb.finish()

wandb:                                                                                


wandb: 
wandb: Run history:
wandb:                     elbo_train ██▇▆▅▄▄▃▃▂▂▂▁▁▁▁
wandb:                elbo_validation █▇▆▅▄▄▃▃▂▂▂▂▁▁▁▁
wandb:                          epoch ▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
wandb:                kl_global_train ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:           kl_global_validation ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:                 kl_local_train ▇█▇▆▅▄▄▃▃▂▂▂▁▁▁▁
wandb:            kl_local_validation █▇▆▅▄▄▃▃▃▂▂▂▁▁▁▁
wandb:      reconstruction_loss_train █▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁
wandb: reconstruction_loss_validation █▆▅▄▄▃▂▂▂▂▂▁▁▁▁▁
wandb:                 train_accuracy ▁▅▅▆▆▇▇▇▇▇▇█████
wandb:        train_calibration_error █▄▃▂▂▂▂▂▂▂▁▁▁▁▁▁
wandb:      train_classification_loss █▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁
wandb:                 train_f1_score ▁▅▅▆▆▇▇▇▇▇▇█████
wandb:               train_loss_epoch █▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁
wandb:                train_loss_step ▃▅▅▄▄▂▃██▅▆▄▄▄▃▆▅▃▆▁▂█▁▅▅▅▄▆▆▅█▄▄▅▂▆▃▅▆▆
wandb:            trainer/global_step ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:            va

wandb: 🚀 View run MAINobj_scANVI_fineTuning_lowLR_noRBCnPlat at: https://wandb.ai/inflammation/inflammation_atlas_R1_scANVI/runs/ye7pjj32/workspace
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)


wandb: Find logs at: ./wandb/run-20240528_215509-ye7pjj32/logs
